In [1]:
pip install torch transformers language-tool-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00


In [2]:
!pip install torch transformers pandas numpy
!pip install textblob

In [10]:
# Install Required Packages
!pip install vaderSentiment
!pip install sentence-transformers
!pip install textblob
!pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 8.2 MB/s eta 0:00:00


In [15]:
import random
import re
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel


emotional_data_path = '/content/mydata.csv'
quotes_data_path = '/content/engqqes.txt'

emotional_df = pd.read_csv(emotional_data_path)


with open(quotes_data_path, 'r', encoding='utf-8') as file:
    quotes_text_data = file.readlines()


negative_words = set([
    "die", "death", "sad", "fear", "pain",
    "hurt", "suffer", "alone", "failure",
    "cry", "darkness", "lonely", "lost"
])


positive_endings = [
    "keep going.", "you are stronger than you think.",
    "there's always hope.", "you are not alone.",
    "believe in yourself.", "you matter.",
    "your story isn't over.", "tomorrow is a new day."
]


def classify_emotion(row):
    worry = row['13. On a scale of 1 to 5, how much are you bothered by worries?']
    depressed = row['18. How often do you feel depressed or down?']
    interest_fluctuation = row['19. On a scale of 1 to 5, how frequently does your interest in daily activities fluctuate?']
    sleep_issues = row['20. On a scale of 1 to 5, how often do you face issues regarding sleep?']


    if depressed <= 2 and worry <= 2 and sleep_issues <= 2:
        return "Happy"
    elif worry >= 4 or sleep_issues >= 4:
        return "Anxious"
    elif depressed >= 4 and interest_fluctuation >= 4:
        return "Depressed"
    else:
        return "Neutral"


emotional_df['Emotional_State'] = emotional_df.apply(classify_emotion, axis=1)


def organize_quotes_by_theme(quotes_text_data):
    themes = {
        "Happy": [],
        "Anxious": [],
        "Depressed": [],
        "Neutral": []
    }


    happy_keywords = ["joy", "happiness", "gratitude", "positivity", "love", "success"]
    anxious_keywords = ["calm", "peace", "courage", "hope", "strength", "patience"]
    depressed_keywords = ["healing", "perseverance", "strength", "resilience", "hope", "growth"]
    neutral_keywords = ["motivation", "life", "dream", "future", "success", "change"]

    for quote in quotes_text_data:
        lower_quote = quote.lower()
        if any(keyword in lower_quote for keyword in happy_keywords):
            themes["Happy"].append(quote)
        elif any(keyword in lower_quote for keyword in anxious_keywords):
            themes["Anxious"].append(quote)
        elif any(keyword in lower_quote for keyword in depressed_keywords):
            themes["Depressed"].append(quote)
        else:
            themes["Neutral"].append(quote)

    return themes


themed_quotes = organize_quotes_by_theme(quotes_text_data)


positive_seed_phrases = {
    "Happy": [
        "Today is a beautiful day because",
        "Happiness is found in",
        "The joy of life is in",
        "Keep smiling because",
        "You deserve all the happiness because"
    ],
    "Anxious": [
        "No matter how hard it seems,",
        "You are stronger than your anxiety because",
        "Take a deep breath and remember that",
        "Courage is not the absence of fear but",
        "You are not alone because"
    ],
    "Depressed": [
        "Even the darkest night will end because",
        "You are stronger than you feel because",
        "Healing is not linear, but",
        "Your story isn't over because",
        "There is hope even when"
    ],
    "Neutral": [
        "Believe in yourself because",
        "Success comes to those who",
        "The future is full of possibilities because",
        "Keep moving forward because",
        "Your dreams are worth"
    ]
}


tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


def enhance_with_llm(seed_phrase, max_new_tokens=50, temperature=0.7):
    input_ids = tokenizer.encode(seed_phrase, return_tensors='pt')
    output = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)


    end_tokens = ['.', '!', '?']
    for token in end_tokens:
        if token in generated_text:
            generated_text = generated_text.split(token)[0] + token
            break

    return generated_text


def get_quote_by_emotion(emotion, themed_quotes):
    seed_phrase = random.choice(positive_seed_phrases[emotion])
    enhanced_quote = enhance_with_llm(seed_phrase)
    return enhanced_quote


def generate_bulk_quotes(emotional_df, themed_quotes, num_quotes=3):
    emotional_states = emotional_df['Emotional_State'].unique()
    bulk_quotes = {}

    for state in emotional_states:
        quotes = [get_quote_by_emotion(state, themed_quotes) for _ in range(num_quotes)]
        bulk_quotes[state] = quotes

    return bulk_quotes


bulk_quotes = generate_bulk_quotes(emotional_df, themed_quotes, num_quotes=3)


for state, quotes in bulk_quotes.items():
    print(f"\nEmotional State: {state}")
    for i, quote in enumerate(quotes, 1):
        print(f"{i}. {quote}")

from textblob import TextBlob


def evaluate_quote_subjectivity(quotes):
    subjectivity_scores = []
    for quote in quotes:
        analysis = TextBlob(quote)
        subjectivity_scores.append(analysis.sentiment.subjectivity)
    return subjectivity_scores


for state, quotes in bulk_quotes.items():
    print(f"\nEmotional State: {state}")
    subjectivity_scores = evaluate_quote_subjectivity(quotes)
    for i, score in enumerate(subjectivity_scores, 1):
        print(f"Quote {i} Subjectivity Score: {score:.2f}")


Emotional State: Anxious
1. You are stronger than your anxiety because your mind is always moving.
2. Take a deep breath and remember that this is a game that requires you to be willing to play with an open mind and a huge amount of patience.
3. Courage is not the absence of fear but the presence of purpose.

Emotional State: Depressed
1. Even the darkest night will end because of the dark power of a single person.
2. You are stronger than you feel because you are strong in these areas, and if you want to go back, you must be stronger.
3. Your story isn't over because you have a bunch of money.

Emotional State: Happy
1. Keep smiling because the rest of the world can see it.
2. You deserve all the happiness because you're a good man and we're all happy because we are good".
3. Today is a beautiful day because of today's amazing people.

Emotional State: Neutral
1. Success comes to those who seek to achieve a moral or religious conclusion within the framework of a Christian worldview.
